In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = '../input/ms-gen-train-test-data/train_temp_100.csv'
test_path =  '../input/ms-gen-train-test-data/test_temp_100.csv'
val_path = '../input/ms-gen-train-test-data/val_temp_100.csv'

In [8]:
oof_train_path = 'oof_train_temp_100.csv'
oof_test_path = 'oof_test_temp_100.csv'

In [9]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 824 ms, sys: 135 ms, total: 959 ms
Wall time: 1.51 s


In [10]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [11]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 2000), (804, 2000))

In [12]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [13]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [14]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [15]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [16]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [17]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [18]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [19]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [20]:
train_set_100.shape, test_set_100.shape

((1059, 2000), (804, 2000))

In [21]:
assert (train_set_100.columns == test_set_100.columns).all()

In [22]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 250.45it/s]


(1059, 101)


100%|██████████| 100/100 [00:01<00:00, 93.63it/s]


(1059, 1900)


100%|██████████| 100/100 [00:00<00:00, 100.07it/s]

(1059, 2200)


In [23]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 244.28it/s]


(804, 101)


100%|██████████| 100/100 [00:01<00:00, 97.99it/s]


(804, 1900)


100%|██████████| 100/100 [00:00<00:00, 102.25it/s]

(804, 2200)


In [24]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [25]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [26]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [27]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [28]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],5)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.29167785957179027


100%|██████████| 101/101 [00:00<00:00, 532861.26it/s]


27
0.27294936868956277


100%|██████████| 1900/1900 [00:00<00:00, 1602488.96it/s]


53
0.25848754126462253


...

4995:	learn: 0.0027297	total: 1m 40s	remaining: 80.7ms
4999:	learn: 0.0027270	total: 1m 40s	remaining: 0us
fold Loss 17:  0.07930148053068829
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6887570	total: 30.6ms	remaining: 2m 32s
999:	learn: 0.0318375	total: 20.3s	remaining: 1m 21s
1998:	learn: 0.0121185	total: 40.9s	remaining: 1m 1s
2997:	learn: 0.0066212	total: 1m	remaining: 40.6s
3996:	learn: 0.0041207	total: 1m 21s	remaining: 20.4s
4995:	learn: 0.0029151	total: 1m 41s	remaining: 81.2ms
4999:	learn: 0.0029115	total: 1m 41s	remaining: 0us


19it [32:12, 101.83s/it]

fold Loss 18:  0.10362806945490331
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6890493	total: 59.2ms	remaining: 4m 55s
999:	learn: 0.0308154	total: 20.2s	remaining: 1m 20s
1998:	learn: 0.0121666	total: 40.4s	remaining: 1m
2997:	learn: 0.0067910	total: 1m	remaining: 40.4s
3996:	learn: 0.0043644	total: 1m 20s	remaining: 20.2s


20it [33:54, 101.70s/it]

4995:	learn: 0.0028214	total: 1m 40s	remaining: 80.8ms
4999:	learn: 0.0028179	total: 1m 40s	remaining: 0us
fold Loss 19:  0.10843318369811744
---------------------------------------------------------------------------
Training Loss: 0.008007763765349576
Loss for sulfide:
0.06204412086515151 0.04895787022097398
AGG Loss: [0.13495215 0.09446428 0.14257785 0.20373867 0.0161171  0.12598981
 0.19941611 0.17067399 0.16221333 0.06204412]
AGG Loss mean: 0.13121874205495992


In [29]:
print(np.array(agg_loss).mean())

0.13121874205495992


In [30]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [31]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003186,0.001625,0.015222,0.008021,0.000124,0.005014,0.015674,0.065599,0.005056,0.000514
767,NaN,0.037774,0.006636,0.002606,0.017867,0.000051,0.004702,0.008337,0.090751,0.017376,0.000845
768,NaN,0.865090,0.087814,0.001996,0.032132,0.000345,0.005858,0.832951,0.911689,0.026030,0.000539
769,NaN,0.012766,0.004569,0.029337,0.018317,0.000063,0.978758,0.020006,0.003073,0.964654,0.000490
770,NaN,0.007582,0.004777,0.014956,0.652107,0.000088,0.996643,0.908900,0.003113,0.007083,0.000210
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.005788,0.001495,0.027285,0.008847,0.000053,0.002864,0.008892,0.040025,0.006405,0.000266
1055,NaN,0.005286,0.007400,0.010726,0.045013,0.000462,0.013761,0.009130,0.886591,0.002406,0.000673
1056,NaN,0.004528,0.008052,0.002721,0.014975,0.998319,0.001675,0.005471,0.002367,0.003441,0.000325
1057,NaN,0.006730,0.934666,0.009120,0.047257,0.000416,0.004547,0.036154,0.009892,0.003446,0.000431


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003895,0.001882,0.020722,0.009742,0.000130,0.006481,0.020051,0.210322,0.005038,0.000530
767,NaN,0.084150,0.002395,0.003134,0.021694,0.000058,0.006166,0.014417,0.404384,0.025376,0.000896
768,NaN,0.426356,0.679323,0.002594,0.078947,0.000428,0.007233,0.511968,0.776514,0.076091,0.000559
769,NaN,0.029244,0.004944,0.069207,0.023138,0.000053,0.944690,0.026789,0.003711,0.909956,0.000304
770,NaN,0.006368,0.006655,0.016030,0.108176,0.000069,0.996197,0.816712,0.003622,0.009334,0.000186
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.009151,0.001031,0.031883,0.009630,0.000057,0.003039,0.010715,0.095364,0.009364,0.000266
1055,NaN,0.006603,0.009023,0.011007,0.058269,0.000334,0.032713,0.010479,0.340599,0.002523,0.000635
1056,NaN,0.005993,0.011765,0.003159,0.015971,0.998135,0.001861,0.005486,0.002478,0.004251,0.000361
1057,NaN,0.010016,0.839258,0.010415,0.092333,0.000173,0.009322,0.053916,0.010333,0.004854,0.000754


In [32]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.13464996912904922
carbonate 0.0939746088848205
chloride 0.16129026907794142
iron_oxide 0.24873268525319875
oxalate 0.0018173834077362665
oxychlorine 0.12084660355807933
phyllosilicate 0.18583324741670673
silicate 0.1563923826136839
sulfate 0.1825180699474463
sulfide 0.05936032511676919
0.13454155444054314


In [33]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.024580764258541953
carbonate 0.015507957426618818
chloride 0.028970826373948207
iron_oxide 0.05864606881885235
oxalate 0.0005551691631746356
oxychlorine 0.018628899102664305
phyllosilicate 0.046999470365294796
silicate 0.032027387897061
sulfate 0.026228106651201673
sulfide 0.004162762999363194
0.025630741305672094


In [34]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [35]:
oof_train_df.to_csv(oof_train_path, index=False)
oof_test_df.to_csv(oof_test_path, index=False)